# 药物数据整理

In [1]:
import pandas as pd


In [11]:
df_gsva = pd.read_csv('../data/OmicsExpressionProteinCodingGenesTPMLogp1-23Q2_GSVA.csv', index_col=0).T
df_gsva.head()

Unnamed: 0,REACTOME_INTERLEUKIN_6_SIGNALING,REACTOME_APOPTOSIS,REACTOME_HEMOSTASIS,REACTOME_INTRINSIC_PATHWAY_FOR_APOPTOSIS,REACTOME_MAPK3_ERK1_ACTIVATION,REACTOME_TRANSLESION_SYNTHESIS_BY_Y_FAMILY_DNA_POLYMERASES_BYPASSES_LESIONS_ON_DNA_TEMPLATE,REACTOME_RECOGNITION_OF_DNA_DAMAGE_BY_PCNA_CONTAINING_REPLICATION_COMPLEX,REACTOME_TRANSLESION_SYNTHESIS_BY_POLH,REACTOME_RECOGNITION_AND_ASSOCIATION_OF_DNA_GLYCOSYLASE_WITH_SITE_CONTAINING_AN_AFFECTED_PURINE,REACTOME_DISPLACEMENT_OF_DNA_GLYCOSYLASE_BY_APEX1,...,REACTOME_NFE2L2_REGULATES_PENTOSE_PHOSPHATE_PATHWAY_GENES,REACTOME_GROWTH_HORMONE_RECEPTOR_SIGNALING,REACTOME_ANTIGEN_PROCESSING_UBIQUITINATION_PROTEASOME_DEGRADATION,REACTOME_CLASS_I_MHC_MEDIATED_ANTIGEN_PROCESSING_PRESENTATION,REACTOME_ANTIGEN_PRESENTATION_FOLDING_ASSEMBLY_AND_PEPTIDE_LOADING_OF_CLASS_I_MHC,REACTOME_KINESINS,REACTOME_FACTORS_INVOLVED_IN_MEGAKARYOCYTE_DEVELOPMENT_AND_PLATELET_PRODUCTION,REACTOME_ANTIGEN_ACTIVATES_B_CELL_RECEPTOR_BCR_LEADING_TO_GENERATION_OF_SECOND_MESSENGERS,REACTOME_SIGNALING_BY_THE_B_CELL_RECEPTOR_BCR,REACTOME_ION_CHANNEL_TRANSPORT
ACH-001113,0.226843,0.226082,-0.195234,-0.225649,0.297840,-0.202449,-0.261840,0.278529,0.486993,-0.385837,...,0.766659,-0.296538,0.173052,-0.170610,-0.354893,0.171962,-0.183717,-0.271703,0.265658,-0.130904
ACH-001289,-0.311721,0.213534,-0.190056,0.218108,-0.330056,-0.219062,0.284506,0.502348,0.307221,0.245559,...,0.323784,-0.254070,0.265314,0.213994,-0.324233,0.241575,0.138463,-0.486953,0.228773,0.171812
ACH-001339,0.458188,0.309910,0.229690,0.299253,0.439757,0.295875,0.299243,0.510140,-0.220570,-0.384481,...,0.298278,0.350154,0.201492,0.235296,0.421460,-0.186718,0.178213,-0.287388,0.322474,0.254648
ACH-001538,-0.226815,0.321793,0.129742,0.262053,0.325965,-0.224285,-0.261044,-0.312602,0.143985,-0.255538,...,-0.491479,-0.177005,0.184373,0.218121,0.579189,-0.422262,-0.250725,0.213530,0.330256,0.147176
ACH-000242,-0.415626,0.265553,-0.163837,0.183398,-0.416399,0.299901,0.301486,0.415456,-0.376310,-0.388460,...,-0.352138,0.127961,-0.153114,-0.159617,-0.314697,-0.326058,-0.192674,-0.311426,0.189309,-0.163466


In [14]:
# 将df_gsva第一列列名改为Cell_Line_Name
modelIDs = df_gsva.index.values

In [16]:
df_drug_sensitivity = pd.read_csv('../data/drug_sensitivity_with_ModelId_DrugName_CellLineName.csv', index_col=0)
df_drug_sensitivity.head()

,Drug Name,Drug ID,Cell Line Name,IC50
ModelID,,,,
ACH-001711,Camptothecin,1003,PFSK1,-1.462148
ACH-001711,Vinblastine,1004,PFSK1,-4.996545
ACH-001711,Cisplatin,1005,PFSK1,3.179510
ACH-001711,Cytarabine,1006,PFSK1,3.213791
ACH-001711,Docetaxel,1007,PFSK1,-5.534982


In [17]:
# 筛选出df_drug_sensitivity中index含有modelIDs中的元素的行
df_drug_sensitivity_left = df_drug_sensitivity[df_drug_sensitivity.index.isin(modelIDs)]

In [18]:
# 将df_drug_sensitivity_left的Drug Name列取出来，去重
drugs = df_drug_sensitivity_left['Drug Name'].unique()
print(len(drugs))

288


In [21]:
import pandas as pd
import pubchempy as pbc
df_drug = pd.DataFrame(columns=['CID', 'SMILES', 'DRUG_NAME'])

fail = []
i = 1
for name in drugs:
    # drug = pbc.get_compounds(name,'name')
    # as_dataframe=True
    drug_properties = pbc.get_properties(['canonical_smiles'],name,'name',smiles_type='canonical')
    if len(drug_properties) == 0:
        print('===',name,'===')
        fail.append(name)
    if len(drug_properties) > 0:
        drug_properties[0].update({"DRUG_NAME":name})
        df_drug.loc[len(df_drug.index)] = list(drug_properties[0].values())
        # df.insert(pd.DataFrame(drug_properties[0].values()))
        # csv_writer.writerow(list(drug_properties[0].values()))
        print(i)
        print(drug_properties)
        i = i +1

# df_drug['DRUG_NAME_ORIGINAL'] = df_drug_names

df_drug.to_csv('../data/CCLE-GDSC-SMILES.csv',index=False)

print('fail:',fail)

print('done!')

1
[{'CID': 24360, 'CanonicalSMILES': 'CCC1(C2=C(COC1=O)C(=O)N3CC4=CC5=CC=CC=C5N=C4C3=C2)O', 'DRUG_NAME': 'Camptothecin'}]
2
[{'CID': 13342, 'CanonicalSMILES': 'CCC1(CC2CC(C3=C(CCN(C2)C1)C4=CC=CC=C4N3)(C5=C(C=C6C(=C5)C78CCN9C7C(C=CC9)(C(C(C8N6C)(C(=O)OC)O)OC(=O)C)CC)OC)C(=O)OC)O', 'DRUG_NAME': 'Vinblastine'}, {'CID': 241903, 'CanonicalSMILES': 'CCC1(CC2CC(C3=C(CCN(C2)C1)C4=CC=CC=C4N3)(C5=C(C=C6C(=C5)C78CCN9C7C(C=CC9)(C(C(C8N6C)(C(=O)OC)O)OC(=O)C)CC)OC)C(=O)OC)O'}, {'CID': 3823887, 'CanonicalSMILES': 'CCC1(CC2CC(C3=C(CCN(C2)C1)C4=CC=CC=C4N3)(C5=C(C=C6C(=C5)C78CCN9C7C(C=CC9)(C(C(C8N6C)(C(=O)OC)O)OC(=O)C)CC)OC)C(=O)OC)O'}, {'CID': 6710780, 'CanonicalSMILES': 'CCC1(CC2CC(C3=C(CCN(C2)C1)C4=CC=CC=C4N3)(C5=C(C=C6C(=C5)C78CCN9C7C(C=CC9)(C(C(C8N6C)(C(=O)OC)O)OC(=O)C)CC)OC)C(=O)OC)O'}, {'CID': 44417235, 'CanonicalSMILES': 'CCC1(CC2CC(C3=C(CCN(C2)C1)C4=CC=CC=C4N3)(C5=C(C=C6C(=C5)C78CCN9C7C(C=CC9)(C(C(C8N6C)(C(=O)OC)O)OC(=O)C)CC)OC)C(=O)OC)O'}, {'CID': 11104792, 'CanonicalSMILES': 'CCC1(CC2CC(C3=C(